# Question 1
A puzzle has multiple ways of reaching the end solution. Fig. 1 shows a graph that represents all possible routes to the solution. The starting point of the game is represented by A, the solution is represented by S. The other points in the graph are possible intermediary stages.

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://github.com/Awa25/AI-DV_CA2/blob/main/qqimg.png?raw=true", width=500)

### (a) The graph in Fig. 1 is a visualisation of the problem.
#### (i) Identify the differences between a graph and a tree
- Definition: A tree is a special type of graph. A tree is an undirected graph in which any two nodes are connected by exactly one path, and there are no cycles. A graph, in contrast, can have any number of paths connecting nodes and can include cycles.
- Nodes and Edges: In a tree, the number of edges is always one less than the number of nodes. In a generic graph, there can be any number of edges.
- Root: Trees typically have a specific node called the "root" from which hierarchical structures descend. Graphs do not have a designated root.
- Path: In a tree, there's exactly one unique path between any two nodes. In a graph, there can be multiple paths, or even none at all, between two nodes.
- Cycles: Trees cannot have cycles, whereas cycles can be present in graphs.

So Fig. 1 is a graph. It has cycles and multiple paths between nodes indicating it is not a tree, trees don't have cycles, and there are unique paths between any two nodes.

#### (ii) Explain in detail how the graph is an abstraction of the problem.

#### (iii) Identify the advantages of using a visualisation such as the one shown in Fig. 1.

### (b)Demonstrate how Dijkstra’s algorithm would find the shortest path to the solution in Fig.1 through diagrams and written explanation of each stage.

# Question 2
The creator of the puzzle has been told that the A* algorithm is more efficient at finding the shortest path because it uses heuristics. Compare the performance of Dijkstra’s algorithm and the A* search algorithm, referring to heuristics, to find the shortest path to the problem by implementing both algorithms programmatically and comparing the solutions generated in Mark-down. Refer to the complexity of the algorithms and compare the actual time it takes for the solutions to be processed.